In [86]:
import os
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import csv

In [87]:

df_raw = pd.read_csv('charting-m-points.csv', encoding = "ISO-8859-1", dtype='unicode')


In [88]:
df_matches = pd.read_csv('charting-m-matches.csv', encoding = "ISO-8859-1", quotechar=None, quoting=3)

In [89]:
df = df_raw

In [90]:
df_temp = df['match_id'].str.split("-")
first_player = []
second_player = []

for i, match in enumerate(df_temp):
    first_player.append(match[4].replace('_', ' ').strip())
    second_player.append(match[5].replace('_', ' ').strip())

df['first_player'] = first_player
df['second_player'] = second_player

In [93]:
P1_hand = []
P2_hand = []
players = {}
date_dict = {}
for index, row in df_matches.iterrows():
    players[row['Player 1']] = row['Pl 1 hand']
    players[row['Player 2']] = row['Pl 2 hand']
    row['Date'] = str(row['Date'])
    date_dict[row['match_id']] = row['Date'][:4]+'/'+ row['Date'][4:6] + '/' + row['Date'][6:]

player_hands = []
player_hands2 = []
date_list = []
for index, row in df.iterrows():
    p1 = row['first_player'].replace('_', ' ')
    p2 = row['second_player'].replace('_', ' ')
    if p1 in players:
        player_hands.append(players[p1])
    else:
        player_hands.append('R')
    if p2 in players:
        player_hands2.append(players[p2])
    else:
        player_hands2.append('R')
    if row['match_id'] in date_dict:
        date_list.append(date_dict[row['match_id']])
    else:
        date_list.append(None)

df['P1_hand'] = player_hands
df['P2_hand'] = player_hands2
df['Date'] = date_list

df.head(5)

,match_id,Pt,Set1,Set2,Gm1,Gm2,Pts,Gm#,TbSet,TB?,...,isUnforced,isDouble,PtWinner,isSvrWinner,rallyCount,first_player,second_player,P1_hand,P2_hand,Date
0,20220114-M-Adelaide-SF-Marin_Cilic-Thanasi_Kok...,1,0,0,0,0,0-0,1 (1),1,0,...,FALSE,FALSE,2,0,1,Marin Cilic,Thanasi Kokkinakis,R,R,2022/01/14
1,20220114-M-Adelaide-SF-Marin_Cilic-Thanasi_Kok...,2,0,0,0,0,0-15,1 (2),1,0,...,FALSE,FALSE,1,1,1,Marin Cilic,Thanasi Kokkinakis,R,R,2022/01/14
2,20220114-M-Adelaide-SF-Marin_Cilic-Thanasi_Kok...,3,0,0,0,0,15-15,1 (3),1,0,...,TRUE,FALSE,2,0,2,Marin Cilic,Thanasi Kokkinakis,R,R,2022/01/14
3,20220114-M-Adelaide-SF-Marin_Cilic-Thanasi_Kok...,4,0,0,0,0,15-30,1 (4),1,0,...,FALSE,FALSE,1,1,11,Marin Cilic,Thanasi Kokkinakis,R,R,2022/01/14
4,20220114-M-Adelaide-SF-Marin_Cilic-Thanasi_Kok...,5,0,0,0,0,30-30,1 (5),1,0,...,FALSE,FALSE,1,1,5,Marin Cilic,Thanasi Kokkinakis,R,R,2022/01/14


In [94]:
tennis_serve_direction = {'4':'out wide', 
                   '5':'body',
                   '6':'down the T', 
                   '0':'unknown'
                  }
tennis_serve_faults = {'n':'net',
                'w':'wide',
                'd':'deep',
                'x':'both wide and deep',
                'g':'foot faults',
                'e':'unknown',
                '!':'shank',
                'V':'time violation',
                'c':'let',
                '+':'serve and volley',
                '*':'Ace',
                '#':'forced error',
                '@':'unforced error'
              }
tennis_shot_type = {
    'f':'forehand groundstroke',
    'b':'backhand groundstroke',
    'r':'forehand slice',
    's':'backhand slice',
    'v':'forehand volley',
    'z':'backhand volley',
    'o':'standard overhead/smash',
    'p':'"backhand" overhead/smash',
    'u':'forehand drop shot',
    'y':'backhand drop shot',
    'l':'forehand lob',
    'm':'backhand lob',
    'h':'forehand half-volley',
    'i':'backhand half-volley',
    'j':'forehand swinging volley',
    'k':'backhand swinging volley',
    't':'all trick shots, including behind-the-back, between-the-legs, and "tweeners."',
    'q':'any unknown shot',
    '*':'winner'

}
tennis_shot_direction = {
    '1':'to a right-hander''s forehand side or left-hander''s backhand side',
    '2':'down the middle of the court',
    '3':'to a right-hander''s backhand side or left-hander''s forehand side',
    '0':'unknown'
}
tennis_shot_depth = {
    '7':'within the service boxes',
    '8':'behind the service line, but closer to the service line than the baseline',
    '9':'closer to the baseline than the service line'
}
tennis_court_position = {
    '^':'Stop volleys/drop volleys',
    ';':'hits the net cord',
    '-':'shot took place at the net',
    '=':'shot took place at the baseline',
    '+':'approach'
}

In [95]:
rallyCount = df['rallyCount']
first_serve = df['1st']
second_serve = df['2nd']
isforced = df['isForced']
isace = df['isAce']
isunforced = df['isUnforced']
isdouble = df['isDouble']

serve_faults = []
serve_direction = []
shot_depth = []
shot_type = []
shot_direction = []
court_position = []
first_serve_list = []
all_serve_pieces = []
#print(rallyCount[:10], first_serve[:10], second_serve[:10], isace[:10], isforced[:10], isunforced[:10], isdouble[:10])
for serve1, serve2, rally in zip(first_serve, second_serve, rallyCount):
  serve = serve1
  #print(rally)
  if str(rally).isdigit() and len(serve1) < int(rally):
    first_serve_list.append(False)
    serve = serve2
  else:
    first_serve_list.append(True)

  #rally_seq = []
  res = []
  serve_faults_list = []
  serve_direction_list = []
  shot_depth_list = []
  shot_type_list = []
  shot_direction_list = []
  court_position_list = []
  serve_pieces = []

  index = 0
  for i, s in enumerate(serve):
    if s.isalpha():
        serve_pieces.append(serve[index:i])
        index = i
  if index + 1 != len(serve):
    serve_pieces.append(serve[index:])
  #print(serve_pieces)
  all_serve_pieces.append(serve_pieces)

print(len(all_serve_pieces))
df['serve_pieces'] = all_serve_pieces
df['first_serve'] = first_serve_list
    

577097


In [96]:
print(len(df))

577097


In [97]:
df = df.explode('serve_pieces')

In [98]:
print(len(df))

2910084


In [99]:
serve_faults = []
serve_direction = []
shot_depth = []
shot_type = []
shot_direction = []
court_position = []
first_serve = []
for serve in df['serve_pieces']:
  serve_faults_list = ''
  serve_direction_list = ''
  shot_depth_list = ''
  shot_type_list = ''
  shot_direction_list = ''
  court_position_list = ''

  for s in serve:
    serve_faults_val = tennis_serve_faults.get(s, None)
    serve_direction_val = tennis_serve_direction.get(s, None)
    shot_depth_val = tennis_shot_depth.get(s, None)
    shot_direction_val = tennis_shot_direction.get(s, None)
    court_position_val = tennis_court_position.get(s, None)
    shot_type_val = tennis_shot_type.get(s, None)
    if serve_faults_val:
      serve_faults_list = serve_faults_val
    if serve_direction_val:
      serve_direction_list = serve_direction_val
    if shot_depth_val:
      shot_depth_list = shot_depth_val
    if shot_direction_val:
      shot_direction_list = shot_direction_val
    if shot_type_val:
      shot_type_list = shot_type_val
    if court_position_val:
      court_position_list = court_position_val

  if serve_faults_list == '':
    serve_faults_list = 'Unknown'
  if serve_direction_list == '':
    serve_direction_list = 'Unknown'
  if serve_faults_list == '':
    serve_faults_list = 'Unknown'
  if shot_depth_list == '':
    shot_depth_list = 'Unknown'
  if shot_direction_list == '':
    shot_direction_list = 'Unknown'
  if court_position_list == '':
    court_position_list = 'Unknown'
  ''' 
  print(serve_faults_list)
  print(serve_direction_list)
  print(shot_depth_list)
  print(shot_type_list)
  print(shot_direction_list)
  print(court_position_list)
  print('----------------')
  '''   
  serve_faults.append(serve_faults_list)
  serve_direction.append(serve_direction_list)
  shot_depth.append(shot_depth_list)
  shot_type.append(shot_type_list)
  shot_direction.append(shot_direction_list)
  court_position.append(court_position_list)

df['serve_faults'] = serve_faults
df['serve_direction'] = serve_direction
df['shot_depth'] = shot_depth
df['shot_type'] = shot_type
df['shot_direction'] = shot_direction
df['court_position'] = court_position


In [104]:
print(df.iloc[10])

match_id           20220114-M-Adelaide-SF-Marin_Cilic-Thanasi_Kok...
Pt                                                                 4
Set1                                                               0
Set2                                                               0
Gm1                                                                0
Gm2                                                                0
Pts                                                            15-30
Gm#                                                            1 (4)
TbSet                                                              1
TB?                                                                0
TBpt                                                             NaN
Svr                                                                1
Ret                                                                2
Serving                                                           MC
1st                               

In [ ]:
del df['1stSV']
del df['2ndSV']
del df['serve_pieces']
del df['Unnamed: 0']

In [103]:
df.to_csv('data_features.csv')

In [81]:
print(print(df.iloc[1]['match_id']))

20220114-M-Adelaide-SF-Marin_Cilic-Thanasi_Kokkinakis_
None
